In [2]:
# Load model directly
from transformers import AutoImageProcessor, AutoModel

processor = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")
model = AutoModel.from_pretrained("google/vit-base-patch16-224-in21k")

In [3]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
from PIL import Image
import os

# Define a dummy dataset
class DummyDataset(Dataset):
    def __init__(self, num_samples=100, img_size=(224, 224)):
        self.num_samples = num_samples
        self.img_size = img_size
        # Generate random data between 0 and 1
        self.data = [(torch.rand(3, *img_size), torch.randint(0, 10, (1,)).item()) for _ in range(num_samples)]

    def __len__(self):
        return self.num_samples

    def __getitem__(self, idx):
        return self.data[idx]

# Load the pre-trained model and processor
model_name = "google/vit-base-patch16-224-in21k"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name, num_labels=10) # Assuming 10 classes for the dummy dataset

# Create dummy dataset and dataloader
train_dataset = DummyDataset()
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
loss_fn = torch.nn.CrossEntropyLoss()

# Fine-tuning loop (simplified)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

model.train()
for epoch in range(3):  # Fine-tune for 3 epochs
    for inputs, labels in train_dataloader:
        # Preprocess the inputs
        processed_inputs = processor(images=inputs, return_tensors="pt")
        pixel_values = processed_inputs.pixel_values.to(device)

        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(pixel_values=pixel_values, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

print("Fine-tuning complete.")

# You can now save the fine-tuned model
# model.save_pretrained("./fine_tuned_model")
# processor.save_pretrained("./fine_tuned_model")

ModuleNotFoundError: No module named 'torchvision'